In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import math


In [3]:
# Load data of features
songs = pd.read_csv("data_100_genre.csv", sep=',')

# Only get rows with complete preview_url
songs_complete = songs[songs[['preview_url']].notnull().all(1)]

# Keep necessary variables
songs_clean = songs_complete[['track_name', 'track_genre',
                     'key', 'loudness', 'mode', 'tempo', 'time_signature', 
                     'energy', 'danceability', 'speechiness', 'acousticness',
                     'instrumentalness', 'liveness', 'valence']]


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder

# Encode outcome ordinally
enc = OrdinalEncoder()
songs_clean[['track_genre']] = enc.fit_transform(songs_clean[['track_genre']])

# Train-test split
train, test = train_test_split(songs_clean, test_size=0.25, random_state=5)

# Split outcome from predictors
train_y = train['track_genre']
train_X = train.drop(columns=['track_genre', 'track_name'])
train_names = train['track_name']
test_y = test['track_genre']
test_X = test.drop(columns=['track_genre', 'track_name'])
test_names = test['track_name']



/var/folders/ny/_mh57ksd0mj1ssyt6hg6_vnw0000gn/T/ipykernel_57932/2565869987.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  songs_clean[['track_genre']] = enc.fit_transform(songs_clean[['track_genre']])


In [5]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(max_depth=2, n_estimators=300, random_state=3,
                                oob_score=True)



In [6]:
# Pipelines for numeric and categorical variables
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import cross_val_score 
from sklearn.pipeline import Pipeline 
from sklearn.compose import ColumnTransformer, make_column_selector

## Seperate pipelines for numeric vs. cat vars
num_transformer = Pipeline([("scaler", MinMaxScaler())])
cat_transformer = Pipeline([("encoder", OneHotEncoder(sparse=False, handle_unknown='ignore'))])

## Column ids
cat_cols = ['key', 'mode']
num_cols = ['loudness', 'tempo', 'time_signature', 
            'energy', 'danceability', 'speechiness', 'acousticness',
            'instrumentalness', 'liveness', 'valence']

## Pre-processor
preprocessor = ColumnTransformer([
    ('num', num_transformer, num_cols),
    ('cat', cat_transformer, cat_cols)
])

## Final pipeline
pipe = Pipeline([
  ('preprocessor', preprocessor),
  ('model', forest)
])

pipe.fit(train_X, train_y)


/Users/anhvu/Documents/Classwork/Spring 2023/STA395/song-genre-predictor/.venv/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   MinMaxScaler())]),
                                                  ['loudness', 'tempo',
                                                   'time_signature', 'energy',
                                                   'danceability',
                                                   'speechiness',
                                                   'acousticness',
                                                   'instrumentalness',
                                                   'liveness', 'valence']),
                                                 ('cat',
                                                  Pipeline(steps=[('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  ['key', 'mode'])])),
                ('model',
                 RandomForestClassifier(max_depth=2, n_estimators=300,
                                        oob_score=True, random_state=3))])

In [8]:
# Out-of-sample accuracy
pipe.named_steps['model'].oob_score_

0.32555555555555554

In [9]:
# Evaluate on test data
pipe.score(test_X, test_y)

0.35